# MNIST Embeddings
-------------

## Imports and Hyperparameters
---------------

In [ ]:
import sys
#Change path specificly to your directories
sys.path.insert(1, '/home/codahead/Fishial/FishialReaserch')

import torch
import torch.nn as nn
import os
import numpy as np
from torchvision import transforms
from torchvision import models
from module.classification_package.src.dataset import FishialDataset
from module.classification_package.src.model import EmbeddingModel, Backbone, Model

# os.chdir('..'); 
print(os.getcwd())

from module.classification_package.deep_features import DeepFeatures
BATCH_SIZE = 128
DATA_FOLDER = r'/home/codahead/Fishial/FishialReaserch/datasets/cutted_fish'
IMGS_FOLDER = 'Outputs/MNIST/Images'
EMBS_FOLDER = 'Outputs/MNIST/Embeddings'
TB_FOLDER = 'Outputs/Tensorboard'
EXPERIMENT_NAME = 'FISHIAL_RESNET18'
DEVICE = 'cpu'

## Create Dataloader
----------------------------

In [ ]:
def stack(tensor, times=3):
    return(torch.cat([tensor]*times, dim=0))



tfs = transforms.Compose([
            transforms.Resize([224, 224]),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])

FISHIAL_data = FishialDataset(
        json_path="data_train.json",
        root_folder=DATA_FOLDER,
        transform=tfs)

data_loader = torch.utils.data.DataLoader(FISHIAL_data,
                                          batch_size=BATCH_SIZE,
                                          shuffle=True)

## Initialize Pretrained Model
-------------------

In [ ]:
resnet18 = models.resnet18(pretrained=True)
resnet18.fc = nn.Identity()

backbone = Backbone(resnet18)
embedding_model = EmbeddingModel(backbone)
embedding_model.load_state_dict(torch.load('/home/codahead/Fishial/output/final_qudruplet_cross_entropy_0.95_130400.0.ckpt'))
embedding_model.eval()

## Initialize Tensorboard Logging Class
--------------

In [ ]:
DF = DeepFeatures(model = embedding_model, 
                  imgs_folder = IMGS_FOLDER, 
                  embs_folder = EMBS_FOLDER, 
                  tensorboard_folder = TB_FOLDER, 
                  experiment_name=EXPERIMENT_NAME)

## Write Embeddings to Tensorboard
------------

In [ ]:
all_preds = []
all_label = {}
data_labels = FISHIAL_data.library_name
for batch in data_loader:
    batch_imgs, labels = batch
    llabels = [data_labels[i]['label'] for i in labels.detach().numpy()]
    labels_name = DF.write_embeddings(x = batch_imgs.to(DEVICE))
    print(len(labels_name) == len(llabels))
    for i in range(len(labels_name)):
        all_label.update({
            labels_name[i]: llabels[i]
        })
    print(len(llabels))
DF.create_tensorboard_log(all_label)